In [33]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder 
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense

from keras import backend as K


#%%
data_x = np.loadtxt('input_x.csv', dtype=float)
data_y = np.loadtxt('input_y.csv', dtype=float)

data_x_test = np.loadtxt('input_x_test.csv', dtype=float)
data_y_test = np.loadtxt('input_y_test.csv', dtype=float)

print(data_x.shape, data_y.shape, data_x_test.shape, data_y_test.shape)
#%%
print(data_x[5], data_y[5])
#%%


(142, 16) (142, 4) (42, 16) (42, 4)
[0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1.] [0. 0. 0. 1.]


In [45]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [46]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(16,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax)
])

model.add(Dense(4, input_dim=4,
                kernel_regularizer=regularizers.l2(0.01)))
#%%
model.summary()
#%%
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy', f1_m,precision_m, recall_m])

model.fit(data_x, data_y, epochs=300)

#%%
loss, accuracy, f1_score, precision, recall = model.evaluate(data_x_test, data_y_test)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_19 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 4)                 68        
_________________________________________________________________
dense_43 (Dense)             (None, 4)                 20        
Total params: 360
Trainable params: 360
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
142/142 [==============================] - 0s 1ms/sample - loss: 0.2770 - acc: 0.7042 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 2/300
142/142 [==============================] - 0s 61us/sample - loss: 0.2607 - 

In [0]:
model.save('sf_nn.h5')